# Использование yaml

Импортируем необходимые библиотеки

In [1]:
import yaml

Опишем конфигурацию, описывающую создание персонажа

In [2]:
hero_yaml = '''
--- !Character
factory:
  !factory assassin
name:
  7NaGiBaToR7
'''

Используем абстрактную фабрику, использование которой будем конфигурировать

In [3]:
class HeroFactory:
    @classmethod
    def create_hero(Class, name):
        return Class.Hero(name)
    
    @classmethod
    def create_weapon(Class):
        return Class.Weapon()
    
    @classmethod
    def create_spell(Class):
        return Class.Spell()

In [4]:
class WarriorFactory(HeroFactory):
    class Hero:
        def __init__(self, name):
            self.name = name
            self.weapon = None
            self.armor = None
            self.spell = None

        def add_weapon(self, weapon):
            self.weapon = weapon

        def add_spell(self, spell):
            self.spell = spell

        def hit(self):
            print(f"Warrior {self.name} hits with {self.weapon.hit()}")
            self.weapon.hit()

        def cast(self):
            print(f"Warrior {self.name} casts {self.spell.cast()}")
            self.spell.cast()
              
    class Weapon:
        def hit(self):
            return "Claymore"
        
    class Spell:
        def cast(self):
            return "Power"
        
    
class MageFactory(HeroFactory):
    class Hero:
        def __init__(self, name):
            self.name = name
            self.weapon = None
            self.armor = None
            self.spell = None

        def add_weapon(self, weapon):
            self.weapon = weapon

        def add_spell(self, spell):
            self.spell = spell

        def hit(self):
            print(f"Mage {self.name} hits with {self.weapon.hit()}")
            self.weapon.hit()

        def cast(self):
            print(f"Mage {self.name} casts {self.spell.cast()}")
            self.spell.cast()
            
    class Weapon:
        def hit(self):
            return "Staff"
        
    class Spell:
        def cast(self):
            return "Fireball"
    
        
class AssassinFactory(HeroFactory):
    class Hero:
        def __init__(self, name):
            self.name = name
            self.weapon = None
            self.armor = None
            self.spell = None

        def add_weapon(self, weapon):
            self.weapon = weapon

        def add_spell(self, spell):
            self.spell = spell

        def hit(self):
            print(f"Assassin {self.name} hits with {self.weapon.hit()}")
            self.weapon.hit()

        def cast(self):
            print(f"Assassin {self.name} casts {self.spell.cast()}")
     
    class Weapon:
        def hit(self):
            return "Dagger"
        
    class Spell:
        def cast(self):
            return "Invisibility"

Опишем конструктор, который сможет обрабатывать узел `!factory`. Он будет возвращать соответствующую фабрику.

Конструктор должен принимать 2 аргумента: `loader` и `node`. Объект `loader` — это загрузчик YAML, `node` — узел файла.
Поскольку структура YAML-файла древовидная, то при первичном проходе обработчиком всё содержимое файла помещаеться в древовидную структуру, содержащую информацию файла в текстовом виде. `node` является узлом именно такого текстового дерева, а `loader` — загрузчик умеющий обрабатывать `node`. По итогу, ниже следующий конструктор `factory_constructor` будет являться частью `loader` и будет вызываться им по необходимости.

Для описанного выше YAML-файла: `loader` — загрузчик, «знакомый» с данным конструктором; `node` — хранит текст `assassin` (информация, хранящаяся за именем пользовательского типом `!factory`) и различную дополнительную информацию.
Поскольку `assassin` — простой скаляр, то для его получения (без дополнительной информации) необходимо воспользоваться методом `construct_scalar`. Если бы после `!factory` располагался список, то необходимо было бы воспользоваться методом `construct_sequenc` и т.д.

In [5]:
def factory_constructor(loader, node):
    data = loader.construct_scalar(node)
    if data == "mage":
        return MageFactory
    elif data == "warrior":
        return WarriorFactory
    else:
        return AssassinFactory

Опишем класс `Character`, в который будут загружаться данные из yaml. Определим у него метод `create_hero`, позволяющий создать персонажа в соответствии с конфигурацией.

In [6]:
class Character(yaml.YAMLObject):
    yaml_tag = "!Character"
    
    def create_hero(self):
        hero = self.factory.create_hero(self.name)

        weapon = self.factory.create_weapon()
        spell = self.factory.create_spell()

        hero.add_weapon(weapon)
        hero.add_spell(spell)

        return hero

Присоединим конструктор и создадим персонажа в соответствии с yaml-конфигурацией.

In [7]:
loader = yaml.Loader
loader.add_constructor("!factory", factory_constructor)
hero = yaml.load(hero_yaml).create_hero()
hero.hit()
hero.cast()

Assassin 7NaGiBaToR7 hits with Dagger
Assassin 7NaGiBaToR7 casts Invisibility
